In [1]:
# Quantum-lambda toy prototype (store-based) with teleportation demo
# - Global QuantumStore holds state vector
# - Move-semantics / runtime linearity checks
# - Supports qnew (allocate), H, X, CNOT, meas
# - Demo: teleportation protocol
import numpy as np
from dataclasses import dataclass
from typing import Any, Dict, List, Tuple, Optional, Union
import random

# Basic gates
X = np.array([[0,1],[1,0]], dtype=complex)
Z = np.array([[1,0],[0,-1]], dtype=complex)
H = 1/np.sqrt(2) * np.array([[1,1],[1,-1]], dtype=complex)
CNOT = np.array([[1,0,0,0],
                 [0,1,0,0],
                 [0,0,0,1],
                 [0,0,1,0]], dtype=complex)

def apply_unitary_to_state(state: np.ndarray, unitary: np.ndarray, targets: List[int], n_qubits: int) -> np.ndarray:
    dim = 2**n_qubits
    U_full = np.zeros((dim, dim), dtype=complex)
    for i in range(dim):
        bits = [(i >> k) & 1 for k in range(n_qubits)]
        target_index = 0
        for idx, t in enumerate(targets):
            if bits[t]:
                target_index |= (1 << idx)
        for out_target in range(2**len(targets)):
            amp = unitary[out_target, target_index]
            if amp == 0:
                continue
            out_bits = bits.copy()
            for idx, t in enumerate(targets):
                out_bits[t] = (out_target >> idx) & 1
            j = 0
            for k in range(n_qubits):
                if out_bits[k]:
                    j |= (1 << k)
            U_full[j, i] += amp
    return U_full @ state

class QuantumStore:
    def __init__(self):
        self.n = 0
        self.state = np.array([1.0 + 0j])
        self.handles = {}
        self.next_handle_id = 0

    def allocate_qubit(self, init_state: np.ndarray = None) -> str:
        if init_state is None:
            init_state = np.array([1.0+0j, 0.0+0j])
        self.state = np.kron(self.state, init_state)
        handle = "q" + str(self.next_handle_id)
        self.handles[handle] = self.n
        self.n += 1
        self.next_handle_id += 1
        return handle

    def apply_unitary(self, handle_list: List[str], unitary: np.ndarray):
        targets = [self.handles[h] for h in handle_list]
        self.state = apply_unitary_to_state(self.state, unitary, targets, self.n)

    def measure(self, handle: str) -> int:
        idx = self.handles[handle]
        probs0 = 0.0
        probs1 = 0.0
        dim = 2**self.n
        for i in range(dim):
            bit = (i >> idx) & 1
            amp = self.state[i]
            if bit == 0:
                probs0 += abs(amp)**2
            else:
                probs1 += abs(amp)**2
        r = random.random()
        result = 0 if r < probs0 else 1
        new_state = np.zeros_like(self.state)
        for i in range(dim):
            bit = (i >> idx) & 1
            if bit == result:
                new_state[i] = self.state[i]
        norm = np.linalg.norm(new_state)
        if norm == 0:
            raise RuntimeError('Measurement gave zero-norm post-state (numerical issue)')
        new_state /= norm
        self.state = new_state
        return result

    def debug_state(self):
        out = {}
        dim = 2**self.n
        for i in range(dim):
            amp = self.state[i]
            if abs(amp) > 1e-9:
                out[format(i, '0{}b'.format(self.n))[::-1]] = amp
        return out

# Minimal runtime linear-environment (not used in demo but present)
class LinearEnv:
    def __init__(self, parent: Optional['LinearEnv']=None):
        self.map = {}
        self.parent = parent

    def bind(self, name: str, value: Any):
        if name in self.map:
            raise RuntimeError('Variable ' + name + ' already bound in this linear environment (no contraction). If it\"s classical data, mark it duplicable).')
        self.map[name] = value

    def consume(self, name: str):
        if name not in self.map:
            cur = self
            while cur is not None and name not in cur.map:
                cur = cur.parent
            if cur is None:
                raise RuntimeError('Linear variable ' + name + ' not found (used more than once or never bound)')
            val = cur.map.pop(name)
            return val
        val = self.map.pop(name)
        return val

    def lookup(self, name: str):
        if name in self.map:
            return self.map[name]
        cur = self.parent
        while cur is not None:
            if name in cur.map:
                return cur.map[name]
            cur = cur.parent
        raise RuntimeError('Variable ' + name + ' not found in environment')

# Teleportation demo using store primitives
def run_teleportation_demo():
    store = QuantumStore()
    q_in = store.allocate_qubit()
    # prepare arbitrary test state: apply H to make superposition
    store.apply_unitary([q_in], H)
    a = store.allocate_qubit()
    b = store.allocate_qubit()
    # entangle a-b
    store.apply_unitary([a], H)
    store.apply_unitary([a, b], CNOT)
    print('State before teleportation:', store.debug_state())
    # Bell measurement: CNOT(q_in, a); H(q_in); measure q_in and a
    store.apply_unitary([q_in, a], CNOT)
    store.apply_unitary([q_in], H)
    m1 = store.measure(q_in)
    m2 = store.measure(a)
    print('Measurement results m1=', m1, ' m2=', m2)
    if m2 == 1:
        store.apply_unitary([b], X)
    if m1 == 1:
        store.apply_unitary([b], Z)
    print('State after corrections:', store.debug_state())
    return m1, m2, store

if __name__ == "__main__":
    m1, m2, store = run_teleportation_demo()
    print('Final amplitudes:')
    for bits, amp in store.debug_state().items():
        print(bits, amp)


State before teleportation: {'000': np.complex128(0.4999999999999999+0j), '010': np.complex128(0.4999999999999999+0j), '001': np.complex128(0.4999999999999999+0j), '011': np.complex128(0.4999999999999999+0j)}
Measurement results m1= 1  m2= 0
State after corrections: {'100': np.complex128(0.7071067811865475+0j), '101': np.complex128(-0.7071067811865475+0j)}
Final amplitudes:
100 (0.7071067811865475+0j)
101 (-0.7071067811865475+0j)
